<a href="https://colab.research.google.com/github/francji1/01NAEX/blob/main/R/01NAEX_Exercise_05_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

	
# O1NAEX Exercise 05

In [ ]:
cat(system('sudo apt-get install -y gmp', intern=TRUE), sep = "\n")
cat(system('sudo apt-get install -y partitions', intern=TRUE), sep = "\n")
cat(system('sudo apt-get install -y DoE.base', intern=TRUE), sep = "\n")

list_of_packages <- c("tidyverse","MASS","FrF2", "rsm", "margins") # "agricolae","nortest","lattice","pwr",
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)


In [ ]:
tibble::tibble(
  Package = names(installed.packages()[,3]),
  Version = unname(installed.packages()[,3])
)

##	Problem 6.1
from the chapter 6, D. C. Montgomery DAoE - 8. edition.
	
An engineer is interested in the effects of cutting
speed (A), tool geometry (B), and cutting angle (C) on the life
(in hours) of a machine tool. Two levels of each factor are
chosen, and three replicates of a 23 factorial design are run.
The results are as follows:

In [ ]:
df61 <- read.table("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Problem_6_1.txt",header=TRUE,sep=";")
head(df61)


1. Estimate the factor effects. Which effects appear to be
large?
2. Use the analysis of variance to confirm your conclusions
for part (a).
3. Write down a regression model for predicting tool life
(in hours) based on the results of this experiment.
4. Analyze the residuals. Are there any obvious problems?
5. On the basis of an analysis of main effect and interaction
plots, what coded factor levels of A, B, and C
would you recommend using?
6. Reconsider task 3. Use the regression
model to generate response surface and contour plots of the
tool life response. Interpret these plots. Do they provide insight
regarding the desirable operating conditions for this process?
7. Find the standard error of the factor effects and
approximate 95 percent confidence limits for the factor effects. Do the results of this analysis agree with the conclusions from the analysis of variance?



### **1. Estimate the factor effects. Which effects appear to be large?**

From cube plot the "ToolGeometry" seems to have the largest effect.

In [ ]:
model_allInter <- lm(LifeHours ~ CuttingSpeed*ToolGeometry*CuttingAngle, data = df61)
cubePlot(model_allInter, eff1 = "CuttingSpeed", eff2 = "ToolGeometry", eff3 = "CuttingAngle", round=1)

### **2. Use the analysis of variance to confirm your conclusions for part (a).**

Sum of Squares is largest for variable "ToolGeometry" (SS = 770.7). The second largest SS is for interaction "CuttingSpeed:CuttingAngle" (SS=468.2).

In [ ]:
summary(yield.aov1 <- aov(model_allInter))

### **3. Write down a regression model for predicting tool life (in hours) based on the results of this experiment.**

From previous model I remove all unsignificant variable.

In [ ]:
model_new <- lm(LifeHours ~ ToolGeometry + CuttingAngle + CuttingSpeed:CuttingAngle, data = df61  )
summary(model_new)

Anova summary of new model:

In [ ]:
summary(yield.aov1 <- aov(model_new))

I made ANOVA test to compere these two models. There is no significant difference between these two models (p value = 0.557).

In [ ]:
anova(model_new, model_allInter)

### **4. Analyze the residuals. Are there any obvious problems?**

I would say residuals are OK. I do not see any significant problem. 

In [ ]:
plot(model_new)

### **5. On the basis of an analysis of main effect and interaction plots, what coded factor levels of A, B, and C would you recommend using?**

Recap the summary of final model:

In [ ]:
summary(model_new)

Interaction plot: 

In [ ]:
interaction.plot(x.factor = df61$CuttingAngle, #x-axis variable
                 trace.factor = df61$CuttingSpeed, #variable for lines
                 response = df61$LifeHours, #y-axis variable
                 fun = mean, #metric to plot
                 ylab = "LifeHours",
                 xlab = "CuttingAngle",
                 type = "b",
                 col = c("red", "blue"),
                 lwd = 2, #line width
                 trace.label = "CuttingSpeed",)

I would choose:
*   ToolGeometry = 1... Since ToolGemeotry have positive coefficient in the model
*   CuttingSpeed = -1... Because of interaction plot.
*   CuttingAngle = 1... Because of interaction plot.


The same interaction plot but swapped x-axis and legend.

In [ ]:
interaction.plot(x.factor = df61$CuttingSpeed, #x-axis variable
                 trace.factor = df61$CuttingAngle, #variable for lines
                 response = df61$LifeHours, #y-axis variable
                 fun = mean, #metric to plot
                 ylab = "LifeHours",
                 xlab = "CuttingSpeed",
                 type = "b",
                 col = c("red", "blue"),
                 lwd = 2, #line width
                 trace.label = "CuttingAngle",)

### **6. Reconsider task 3. Use the regression model to generate response surface and contour plots of the tool life response. Interpret these plots. Do they provide insight regarding the desirable operating conditions for this process?**

I would say yes - these plots provide good inside for this process and its desirable operating condition.

In [ ]:
par(mfrow = c(2,2)) 
contour(model_new, ~ ToolGeometry + CuttingAngle + CuttingSpeed, image = TRUE)

The surface plots are for given x and y axis plotted at angles 45, 135, 225 and 315 degrees.



"ToolGeometry" and "CuttingAngle":

In [ ]:
 contour(model_new, ~ ToolGeometry + CuttingAngle, image = TRUE)
persp(model_new, xvar = "ToolGeometry", yvar = "CuttingAngle", theta = c(45, 135, 225, 315))

"CuttingSpeed" and "CuttingAngle":

In [ ]:
contour(model_new, ~ CuttingAngle + CuttingSpeed, image = TRUE)
persp(model_new, xvar = "CuttingSpeed", yvar = "CuttingAngle", theta = c(45, 135, 225, 315))

"ToolGeometry" and "CuttingSpeed":

In [ ]:
contour(model_new, ~ ToolGeometry + CuttingSpeed, image = TRUE)
persp(model_new, xvar = "ToolGeometry", yvar = "CuttingSpeed", theta = c(45, 135, 225, 315))

### **7. Find the standard error of the factor effects and approximate 95 percent confidence limits for the factor effects. Do the results of this analysis agree with the conclusions from the analysis of variance?**

ANOVA:

In [ ]:
summary(model_new)

Standard error:

In [ ]:
stdError <- summary(model_new)$coef[,1]
print(stdError)

95 percent confidence interval:

In [ ]:
confint(model_new)